In [ ]:
import pandas as pd
import numpy as np

# Load the Excel file
file_path = 'coal_supply_chain_india.xlsx'

# View all sheet names (if it's a multi-sheet Excel)
xls = pd.ExcelFile(file_path)
print(xls.sheet_names)

['assets', 'firms', 'ownership', 'supply chain', 'critical infrastructure', 'flood exposure']


In [8]:
# Load a specific sheet (e.g., the first one)
df = pd.read_excel(xls, sheet_name='flood exposure') 


# Identify rows with missing data (i.e., all NaNs except asset_id)
mask_empty = df.iloc[:, 1:].isna().all(axis=1)

# Identify rows with valid data
df_valid = df[~mask_empty]

# Sample with replacement from valid rows for the number of empty rows
sampled_rows = df_valid.sample(n=mask_empty.sum(), replace=True).reset_index(drop=True)

# Copy the original asset_ids from the empty rows
sampled_rows['asset_id'] = df.loc[mask_empty, 'asset_id'].values

# Replace empty rows in the original DataFrame with the sampled ones
df.loc[mask_empty, :] = sampled_rows.values




In [9]:
df

,asset_id,5,10,25,50,100,200,500,1000,flood_protection
0,M0464,9.34,10.15,11.61,13.28,13.57,14.11,14.81,15.19,10.15
1,M0465,0.43,0.55,0.66,0.72,0.79,0.85,0.91,0.98,0.55
2,M0466,2.05,2.25,2.47,2.58,2.60,2.61,2.62,2.62,2.25
3,M0467,2.05,2.25,2.47,2.58,2.60,2.61,2.62,2.62,2.25
4,M0468,0.20,1.07,2.85,2.86,3.92,4.62,5.26,5.67,1.07
...,...,...,...,...,...,...,...,...,...,...
1233,T01164,1.56,1.71,2.43,2.80,3.17,3.44,3.60,3.70,1.71
1234,T01169,5.60,6.32,7.14,7.45,7.99,8.12,8.35,8.56,6.32
1235,T01165,0.43,0.55,0.66,0.72,0.79,0.85,0.91,0.98,0.55
1236,T01176,0.02,1.07,2.00,2.50,3.88,4.08,4.23,4.32,1.07


In [10]:
df_assets = pd.read_excel(xls, sheet_name='assets') 
df_ownership = pd.read_excel(xls, sheet_name='ownership')

# Step 1: Create a mapping from old asset_id to new ones
old_ids = df_assets['asset_id'].tolist()
new_ids = list(range(1, len(df_assets) + 1))
id_map = dict(zip(old_ids, new_ids))

# Step 2: Replace asset_id in all DataFrames
df_assets['asset_id'] = df_assets['asset_id'].map(id_map)
df_ownership['asset_id'] = df_ownership['asset_id'].map(id_map)
df['asset_id'] = df['asset_id'].map(id_map)

# Step 3: Save all sheets back into the original file
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_assets.to_excel(writer, sheet_name='assets', index=False)
    df_ownership.to_excel(writer, sheet_name='ownership', index=False)
    df.to_excel(writer, sheet_name='flood exposure', index=False)

In [5]:
from openpyxl import load_workbook

# Save the updated DataFrame back to the same Excel file
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df.to_excel(writer, sheet_name='flood exposure', index=False)